In [ ]:
import sys
sys.path.append("..")

import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split, KFold, cross_val_score


from main import calibrate_magnetic_wifi_ibeacon_to_position, extract_ibeacon_rssi
from main import extract_magnetic_strength
from io_f import read_data_file
from visualize_f import visualize_trajectory, visualize_heatmap
from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from main import extract_wifi_rssi, extract_wifi_count

from processing import SUBMISSION_SITES, split_sample_sbm_to_dataframe, formatting_for_visualize_waypoint, convert_floor_to_int, concatenate_formatting_waypoints\
,create_magnetic_features

sns.set_style()
%load_ext autoreload
%autoreload 2

Duplicate key in file PosixPath('/Users/takahashigenki/.matplotlib/stylelib/genki.mplstyle'), line 176 ('font.family         : IPAexGothic')


In [ ]:
base = ".."
train_paths = glob.glob("../train/*/*/*.txt")
test_paths = glob.glob("../test/*.txt")
sample_path = f'{base}/train/5cd56b5ae2acfd2d33b58549/5F/5d06134c4a19c000086c4324.txt'
# sample_path = test_paths[1]
print(sample_path)

# sample_mwi = calibrate_magnetic_wifi_ibeacon_to_position([sample_path])
# sample_wifi_rssi = extract_wifi_rssi(sample_mwi)
# sample_wifi_count = extract_wifi_count(sample_mwi)


In [ ]:
sample_data = read_data_file(sample_path)
print(f"N.waypoint: {len(sample_data.waypoint)}, N.wifi: {len(sample_data.wifi)}, N.mag: {len(sample_data.magn)}")
sample_mwi = calibrate_magnetic_wifi_ibeacon_to_position([sample_path])
print(f"N.calibrated_data: {len(sample_mwi)}.")

# waypoint plot
sample_way = sample_data.waypoint[:,1:3]

site = sample_path.split("/")[2]
floorNo = sample_path.split("/")[3]
floor_plan_filename = f'{base}/metadata/{site}/{floorNo}/floor_image.png'


wifi_counts = extract_wifi_count(sample_mwi)
# heat_positions = np.array(list(wifi_counts.keys()))
# heat_values = np.array(list(wifi_counts.values()))
# # filter out positions that no wifi detected
# mask = heat_values != 0
# heat_positions = heat_positions[mask]
# heat_values = heat_values[mask]

# # The heatmap
# visualize_heatmap(heat_positions, 
#                   heat_values, 
#                   floor_plan_filename, 
#                   width_meter, 
#                   height_meter
#                  )

# #uncal
# uncal_mwi = processing.uncalibrate_magnetic_wifi_ibeacon_to_position([sample_path])
# uncal_wifi_rssi = extract_wifi_rssi(uncal_mwi)

# uncal_wifi_counts = extract_wifi_count(uncal_mwi)
# uncal_heat_positions = np.array(list(uncal_wifi_counts.keys()))
# uncal_heat_values = np.array(list(uncal_wifi_counts.values()))
# # filter out positions that no wifi detected
# mask = uncal_heat_values != 0
# uncal_heat_positions = uncal_heat_positions[mask]
# uncal_heat_values = uncal_heat_values[mask]

# # The heatmap
# visualize_heatmap(uncal_heat_positions, 
#                   uncal_heat_values, 
#                   floor_plan_filename, 
#                   width_meter, 
#                   height_meter
#                  )


In [ ]:
print(f"No. of interpolated data: {len(sample_mwi.keys())}\t No. of the raw waypoint:{len(sample_way)}")

1. siteで階層をまたいでデータを結合
2. bsiidをonehot化する, device idを特徴量化する
3. 

In [ ]:
n_wifi_per_waypoint = []
for k, v in sample_mwi.items():
    n_wifi_per_waypoint.append(len(v["wifi"]))
sns.set()
plt.figure(figsize=(8,4))
plt.plot(n_wifi_per_waypoint)
plt.title(f"min: {min(n_wifi_per_waypoint)}")

In [ ]:
fig = plt.figure(figsize=(8,12))
for i, col in enumerate(["magnetic", "ibeacon", "wifi"]):
    n_data = []
    for k, v in sample_mwi.items():
        n_data.append(len(v[col]))
    sns.set()
    ax = fig.add_subplot(3,1,i+1)
    ax.plot(n_data)
    ax.set_title(f"{col} \t min: {min(n_data)}")

# create magnetic feature

In [ ]:
# this function create feature of magnetic values in one interpolated path
def process_magnetic_feature(mwi):
    mag_feature_df = []
    for position_key in mwi:
        # print(f'Position: {position_key}')
        magnetic_data = mwi[position_key]['magnetic']
        magnetic_s = np.mean(np.sqrt(np.sum(magnetic_data[:, 1:4] ** 2, axis=1)))
        magnetic_1_mean = np.mean(magnetic_data[:, 1])
        magnetic_2_mean = np.mean(magnetic_data[:, 2])
        magnetic_3_mean = np.mean(magnetic_data[:, 3])
        magnetic_1_std = np.std(magnetic_data[:, 1])
        magnetic_2_std = np.std(magnetic_data[:, 2])
        magnetic_3_std = np.std(magnetic_data[:, 3])
        magnetic_cov = np.cov(magnetic_data.T)
        tmp_data = [
            magnetic_s,
            magnetic_1_mean,
            magnetic_2_mean,
            magnetic_3_mean,
            magnetic_1_std,
            magnetic_2_std,
            magnetic_3_std,
        ]
        for v in magnetic_cov.reshape(-1): tmp_data.append(v)
        mag_feature_df.append(tmp_data)
    mag_feature_df = pd.DataFrame(mag_feature_df)
    return mag_feature_df

process_magnetic_feature(sample_mwi)




# create waypoint(grand true values)

In [ ]:

def create_interpolate_waypoint(mwi):
    df_gt = pd.DataFrame(mwi.keys(), columns=["x","y"])
    df_gt = pd.concat([df_gt,pd.Series(name="floor", data=[convert_floor_to_int(floorNo) for _ in range(df_gt.shape[0])])], ignore_index=True, axis=1)
    df_gt = pd.concat([df_gt,pd.Series(name="floor", data=[site for _ in range(df_gt.shape[0])])], ignore_index=True, axis=1)
    df_gt.columns = ["x", "y", "floor", "site"]
    return df_gt

create_interpolate_waypoint(sample_mwi)

In [ ]:
pd.concat([create_interpolate_waypoint(sample_mwi), process_magnetic_feature(sample_mwi)], axis=1)

In [ ]:
site

In [ ]:
# select one site
sample = list(SUBMISSION_SITES)[0]
sample_paths = {}
for path in train_paths:
    path_data = path.split("/")    
    if path_data[2] == sample:
        if not path_data[3] in sample_paths.keys():
            sample_paths[f"{path_data[3]}"] = []
        sample_paths[f"{path_data[3]}"].append(path)

# result has structure devided by its floor
# display(sample_paths)


base = ".."
floor = "F2" # because we didnt create matching function of string floor name to the corresponding value;)
site = sample
pathway_paths = sample_paths[floor]


floor_plan_filename = f'{base}/metadata/{site}/{floor}/floor_image.png'

sample_pathway = concatenate_formatting_waypoints(pathway_paths)


# Prepare width_meter & height_meter
json_plan_filename = f'{base}/metadata/{site}/{floor}/floor_info.json'
with open(json_plan_filename) as json_file:
    json_data = json.load(json_file)
    
width_meter = json_data["map_info"]["width"]
height_meter = json_data["map_info"]["height"]
title = site

visualize_trajectory(trajectory = sample_pathway,
                     floor_plan_filename = floor_plan_filename,
                     width_meter = width_meter,
                     height_meter = height_meter,
                     title = title
                    )



# magnetic, wifi, beacon(?)はsitewideで使う

In [ ]:
tmp = create_magnetic_features(sample_paths["F3"], "F3", site)
tmp = tmp.reset_index(drop=True)
tmp

In [ ]:
mg_df = pd.DataFrame()
for floor in sample_paths.keys():
    tmp = create_magnetic_features(sample_paths[floor], floor, site).reset_index(drop=True)
    mg_df = mg_df.append(tmp)
mg_df
    
    

## create LSTM model

In [ ]:

site = mg_df.iloc[0,3]
X = mg_df.iloc[:,4:]
y = mg_df.iloc[:,:3]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

#setting
# def set_seed():
    

# hyper-parameter set

# cross-validation

# epoch

### beacon

In [ ]:
beaconデータを使うのはあまり成功していない。

In [ ]:
sample_beacon = extract_ibeacon_rssi(sample_mwi)
sample_beacon

In [ ]:
sample_mwi